# Topic 2: Recommendation System
### Demo: Underthesea, Gemsim, Cosin Similarity

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [7]:
# Dữ liệu thử nghiệm
df= pd.read_excel('Overview_Companies.xlsx')

In [8]:
df.head()

,id,Company Name,Company Type,Company industry,Company size,Country,Working days,Overtime Policy,Company overview,Our key skills,Why you'll love working here,Location,Href
0,0,1BITLAB Technology,IT Outsourcing,Software Development Outsourcing,1-50 employees,Vietnam,Monday - Friday,No OT,1BITLAB Technology\nCông ty cổ phần công nghệ ...,NaN,NaN,"{'Ha Noi': ['81 Lê Đức Thọ, Nam Từ LIêm, Hà Nộ...",https://itviec.com/companies/1bitlab-technology\n
1,1,1test,IT Product,IT Services and IT Consulting,1-50 employees,South Korea,Monday - Friday,No OT,1test\nIs an innovative educational technology...,NaN,NaN,"{'Ho Chi Minh': ['Remote, District 1, Ho Chi M...",https://itviec.com/companies/1test\n
2,2,52 Entertainment,IT Product,Game,1-50 employees,France,Monday - Friday,No OT,A start-up of French and Vietnamese guys with ...,Our Key Skills\nJava\nGames\nQA QC\nC#\nUnity\...,Be your own boss\nA free lunch and a half day ...,"{'Ho Chi Minh': ['Tầng 7, 23 Trần Cao Vân, Phư...",https://itviec.com/companies/52-entertainment\n
3,3,8Seneca,IT Outsourcing,IT Services and IT Consulting,1-50 employees,Singapore,Monday - Friday,No OT,Pure play IT team extensions B2b exclusive out...,NaN,"Ha Noi\n19F, Coninco Tower 4 Tôn Thất Tùng, Do...","{'Ha Noi': ['19F, Coninco Tower 4 Tôn Thất Tùn...",https://itviec.com/companies/8seneca\n
4,4,Accenture,IT Service and IT Consulting,IT Services and IT Consulting,301-500 employees,Vietnam,Monday - Friday,No OT,A global professional services company with le...,Our Key Skills\nCloud\nLinux\nDevOps\nNetworki...,Hybrid working\nWonderful and Human focus Envi...,"{'Ho Chi Minh': ['9th Floor, Viettel Tower, 28...",https://itviec.com/companies/accenture\n


In [9]:
# Hàm loại bỏ stopwords dùng thư viện cho tiếng Anh
def remove_stopwords_sklearn(text):
    if pd.isna(text):
        return ""
    words = re.findall(r'\b\w+\b', text.lower())
    tokens = [word for word in words if word not in ENGLISH_STOP_WORDS]
    return ' '.join(tokens)

# Tạo cột mới
df['Company_overview'] = df['Company overview'].apply(remove_stopwords_sklearn)

In [10]:
# Tạo nội dung tổng hợp
# Lựa chọn các thông tin cần thiết để đưa vào phần content (giúp đề xuất nội dung)
# Có thể 1 cột hoặc nhiều cột thông tin
# ví dụ ở đây là chọn 1 cột, và chỉ lấy 300 từ đầu tiên trong phần mô tả (dễ quan sát khi làm demo)
df['Content'] = df['Company_overview'].apply(lambda x: ' '.join(x.split()[:300]))

In [11]:
df.head()

,id,Company Name,Company Type,Company industry,Company size,Country,Working days,Overtime Policy,Company overview,Our key skills,Why you'll love working here,Location,Href,Company_overview,Content
0,0,1BITLAB Technology,IT Outsourcing,Software Development Outsourcing,1-50 employees,Vietnam,Monday - Friday,No OT,1BITLAB Technology\nCông ty cổ phần công nghệ ...,NaN,NaN,"{'Ha Noi': ['81 Lê Đức Thọ, Nam Từ LIêm, Hà Nộ...",https://itviec.com/companies/1bitlab-technology\n,1bitlab technology công ty cổ phần công nghệ 1...,1bitlab technology công ty cổ phần công nghệ 1...
1,1,1test,IT Product,IT Services and IT Consulting,1-50 employees,South Korea,Monday - Friday,No OT,1test\nIs an innovative educational technology...,NaN,NaN,"{'Ho Chi Minh': ['Remote, District 1, Ho Chi M...",https://itviec.com/companies/1test\n,1test innovative educational technology compan...,1test innovative educational technology compan...
2,2,52 Entertainment,IT Product,Game,1-50 employees,France,Monday - Friday,No OT,A start-up of French and Vietnamese guys with ...,Our Key Skills\nJava\nGames\nQA QC\nC#\nUnity\...,Be your own boss\nA free lunch and a half day ...,"{'Ho Chi Minh': ['Tầng 7, 23 Trần Cao Vân, Phư...",https://itviec.com/companies/52-entertainment\n,start french vietnamese guys passion games dev...,start french vietnamese guys passion games dev...
3,3,8Seneca,IT Outsourcing,IT Services and IT Consulting,1-50 employees,Singapore,Monday - Friday,No OT,Pure play IT team extensions B2b exclusive out...,NaN,"Ha Noi\n19F, Coninco Tower 4 Tôn Thất Tùng, Do...","{'Ha Noi': ['19F, Coninco Tower 4 Tôn Thất Tùn...",https://itviec.com/companies/8seneca\n,pure play team extensions b2b exclusive outsou...,pure play team extensions b2b exclusive outsou...
4,4,Accenture,IT Service and IT Consulting,IT Services and IT Consulting,301-500 employees,Vietnam,Monday - Friday,No OT,A global professional services company with le...,Our Key Skills\nCloud\nLinux\nDevOps\nNetworki...,Hybrid working\nWonderful and Human focus Envi...,"{'Ho Chi Minh': ['9th Floor, Viettel Tower, 28...",https://itviec.com/companies/accenture\n,global professional services company leading c...,global professional services company leading c...


In [12]:
df[["Company Name"]].head(2)

,Company Name
0,1BITLAB Technology
1,1test


In [13]:
import re

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vector hóa nội dung
vectorizer = TfidfVectorizer(analyzer='word')
tfidf_matrix = vectorizer.fit_transform(df['Content'])

# Tính toán độ tương đồng
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [16]:
df_show = pd.DataFrame(cosine_sim)
df_show

,0,1,2,3,4,5,6,7,8,9,...,468,469,470,471,472,473,474,475,476,477
0,1.000000,0.007987,0.000000,0.000000,0.006290,0.000000,0.000000,0.018313,0.235440,0.002527,...,0.147663,0.004343,0.008124,0.000000,0.003346,0.000000,0.003904,0.187683,0.000000,0.000000
1,0.007987,1.000000,0.000000,0.010852,0.024771,0.036168,0.000000,0.023455,0.000000,0.030121,...,0.000000,0.007607,0.086592,0.028461,0.017261,0.032351,0.009283,0.004897,0.020216,0.032046
2,0.000000,0.000000,1.000000,0.076016,0.003772,0.010936,0.027397,0.012567,0.000000,0.028003,...,0.049428,0.014012,0.022751,0.084461,0.027593,0.020886,0.070656,0.000000,0.000000,0.023429
3,0.000000,0.010852,0.076016,1.000000,0.012876,0.046025,0.020711,0.000000,0.000000,0.047444,...,0.000000,0.028039,0.032114,0.027889,0.012876,0.008486,0.006525,0.007672,0.020583,0.039026
4,0.006290,0.024771,0.003772,0.012876,1.000000,0.041755,0.016496,0.049768,0.004927,0.063018,...,0.000000,0.038948,0.041404,0.041681,0.028838,0.022226,0.034438,0.003197,0.006283,0.087792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,0.000000,0.032351,0.020886,0.008486,0.022226,0.040126,0.012070,0.025826,0.000000,0.025603,...,0.003714,0.008607,0.041236,0.050290,0.026481,1.000000,0.005941,0.007242,0.013147,0.049823
474,0.003904,0.009283,0.070656,0.006525,0.034438,0.022697,0.028149,0.011982,0.000000,0.030516,...,0.018065,0.009747,0.031543,0.041760,0.016839,0.005941,1.000000,0.002961,0.041948,0.004210
475,0.187683,0.004897,0.000000,0.007672,0.003197,0.000000,0.000000,0.009493,0.265441,0.009571,...,0.137586,0.001655,0.005215,0.004374,0.000000,0.007242,0.002961,1.000000,0.000000,0.000000
476,0.000000,0.020216,0.000000,0.020583,0.006283,0.032553,0.016550,0.006474,0.000000,0.050164,...,0.004281,0.025081,0.032637,0.043365,0.059470,0.013147,0.041948,0.000000,1.000000,0.023631


In [17]:
# Hàm đề xuất công ty
# # với mỗi sản phẩm, lấy nums sản phẩm tương quan nhất
def get_recommendations(id, cosine_sim=cosine_sim, nums=3):
    idx = df.index[df['id'] == id][0]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:nums+1]  # Lấy 3 sản phẩm tương tự nhất
    cp_indices = [i[0] for i in sim_scores]
    return df.iloc[cp_indices]

In [18]:
# Gọi function
recommendations = get_recommendations(68)
recommendations

68


,id,Company Name,Company Type,Company industry,Company size,Country,Working days,Overtime Policy,Company overview,Our key skills,Why you'll love working here,Location,Href,Company_overview,Content
280,280,MEGAZONE,IT Product,IT Services and IT Consulting,1-50 employees,South Korea,Monday - Friday,No OT,Cloud Innovator\nWe're cloud company. Megazon...,Java\nJ2EE\nCSS\nPHP\nHTML5\nSQL,Professional working environment\nAttractive s...,"{'Ha Noi': ['54 Lieu Giai, Ba Dinh, Ha Noi'], ...",https://itviec.com/companies/megazone\n,cloud innovator cloud company megazone deliver...,cloud innovator cloud company megazone deliver...
253,253,KMS Technology,IT Outsourcing,NaN,1000+ employees,United States,Monday - Friday,No OT,"Agile, Innovative & Excellent Global Technolog...",Java\nJavaScript\n.NET\nQA QC\nNodeJS\nDatabase,Inspired Leadership Team and Talented Colleagu...,"{'Ho Chi Minh': ['02 Tan Vien, ward 2, Tan Bin...",https://itviec.com/companies/kms-technology\n,agile innovative excellent global technology c...,agile innovative excellent global technology c...
387,387,Silicon Prime Labs,IT Outsourcing,Software Development Outsourcing,51-150 employees,United States,Monday - Friday,Extra salary for OT,USA Mobile and Web App Development Company\nOV...,MVC\nASP.NET\n.NET\niOS\nAngularJS\nNodeJS\nRe...,Work with Silicon Valley team\nNewest techs\nM...,"{'Ho Chi Minh': ['68 Bach Dang, Tan Binh, Ho C...",https://itviec.com/companies/silicon-prime-labs\n,usa mobile web app development company overvie...,usa mobile web app development company overvie...


#### Lưu ma trận kết quả consin và đọc lên khi cần đề xuất

In [19]:
# Save cosine_sim to file
import pickle
with open('companies_cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)

# Open and read file to cosine_sim_new
with open('companies_cosine_sim.pkl', 'rb') as f:
    cosine_sim_new = pickle.load(f)
# cosine_sim_new

In [20]:
def get_recommendations(df, id, cosine_sim=cosine_sim, nums=5):
    # Get the index of the hotel that matches the hotel_id
    matching_indices = df.index[df['id'] == id].tolist()
    if not matching_indices:
        print(f"No company found with ID: {id}")
        return pd.DataFrame()  # Return an empty DataFrame if no match
    idx = matching_indices[0]

    # Get the pairwise similarity scores of all companies with that hotel
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the companies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar companies (Ignoring the company itself)
    sim_scores = sim_scores[1:nums+1]

    # Get the hotel indices
    cp_indices = [i[0] for i in sim_scores]

    # Return the top n most similar hotels as a DataFrame
    return df.iloc[cp_indices]

In [21]:
# Example usage (make sure to use a valid ma_san_pham from your DataFrame)
recommendations = get_recommendations(df, 68, cosine_sim=cosine_sim_new, nums=3)

In [22]:
recommendations[["id", "Company Name", "Content"]].head()

,id,Company Name,Content
280,280,MEGAZONE,cloud innovator cloud company megazone deliver...
253,253,KMS Technology,agile innovative excellent global technology c...
387,387,Silicon Prime Labs,usa mobile web app development company overvie...
